## Day 2: Diving Deeper into Pandas
<img src="https://pbs.twimg.com/media/B_1KzLlUYAIFadB.jpg:large" width="200" height="200" />


Welcome to Day 2. Python programmers often organize their code into chunks called modules. A module contains a set of related commands to accomplish a task. Today we are going to learn two of the most useful modules for data manipulation and plotting, `pandas` and `seaborn`. 

First we will use a simple data set on mouse weights taken from males and females of different strains to illustrate how to obtain simple descriptive statistics, group the data, and plot. 

Then, we will looking some imaging statistics data from imaris and explore some more complex operations on the data.

## 2.0 importing in `pandas` and `numpy`

To start, we'll load up two modules with the `import` statement. The first is the `pandas` module, which will let us manipulate 2D tables. We refer to the `pandas` module here as `pd` as an abbreviation. The second module we load is the `numpy` module, mostly for the arithmetic functions built into it. We will do much more with the `numpy` module in Days 3 and 4.

When you load a module using `import`, all of the functions available, such as `np.mean` are now accessible to you. Modules and import statements help programmers avoid naming conflicts because you can use short, straightforward names for functions and variables without worrying that they're already taken. Matlab does not have anything equivalent to Python's module system and therefore can be harder to read. 

In [ ]:
import pandas as pd
import numpy as np

##Once imported, you can refer to the modules by `np`. 
##For example, we can take the mean of a list:
np.mean([2,4,6])

## 2.1 Paths and Loading Data using `read_csv`

Before we even load the data, we need to give Python an idea of where the data is. Our data is in the `data` directory in `day2`. We're going to specify the `path` or the folder location. There are two kinds of paths: *relative*, which is relative to our current directory, and *absolute*: the complete location of the directory. For example, the absolute path to the `data` folder on my Mac is: `/Users/laderast/Code/python_neurobootcamp/day2/data/`. 

The current directory for a Jupyter Notebook is always the directory where the notebook is located in. Since this notebook is in the `day2` directory, that is our current directory. All relative paths will be relative to this directory. 

**Note**: if you can, it's good practice to keep your Jupyter Notebook and your data in the same folder, and use relative paths to access everything. If someone wants to reproduce your analysis, they will be able to rerun everything in your notebook more easily. This makes your code more *reproducible*, which is really important these days. 

Okay, let's get to work! Let's load the data up using the `pd.read_csv` function. The `read_csv` function is a part of the `pandas` module, so we have to include the `pd.` in front of it so the computer knows to look in the `pandas` module to find and use this function. By using `pd.read_csv`, we return what is called a `pandas` `DataFrame`.   Most of our manipulations and plotting are going to be done on this `DataFrame`.

A `DataFrame` can be thought of as a 2D table, but the values within each of the columns must be the same datatype. For example, any entry in the `MouseID` column must be a `string`, and any entry in the `Weight` column must be a `float`.

In [ ]:
mousedata = pd.read_csv("data/mouseData.csv")
##show the whole DataFrame
mousedata

## 2.2 Basic DataFrame manipulations

The first thing to do with our `DataFrame` is to look at the first few rows of the function using `head()`. We often do this just to confirm that we loaded the data correctly (that it has the correct column names).

**Note**: the `.` notation lets us access built in functions that are defined for the `DataFrame`. You can see all of the functions that you can do to a `DataFrame` by typing `mousedata.` into a code cell and hitting the `Tab` key. There's lots of them!

In [ ]:
mousedata.head()

A really useful function that we can immediately call on `mousedata` is `describe`. `describe` will return summary statistics on the numerical variables.

In [ ]:
mousedata.describe()

Another useful thing to note is the `.shape`, which returns the dimensions of the `DataFrame`. This can also be useful in confirming that we loaded the data in correctly.

Note that we don't use `()` after `.shape`. This is because `shape` is a `Property` and not a function. This can be really confusing at first. 

In [ ]:
mousedata.shape

Sometimes we'd like to get the number of rows in our `DataFrame`. Since `mousedata.shape` is a tuple (a special kind of list), we can access it using `[0]`. This can be really useful when we need to do something over all of the rows.

In [ ]:
##get number of rows in mousedata
mousedata.shape[0]

**Question**: How would we get the number of columns?

In [ ]:
##Put your answer below.

## 2.3 A Quick Intro to functions

Before we can move forward, we need to talk about functions for a little bit. You can think of a function as a bit of reusable code. The important thing is that you need to define the inputs (what goes into the function) and the output (what comes out of the function).

Try and run the following function. What does it do?

In [ ]:
##always begin with "def" when defining a new function, 
##have an interface defined in the "()", 
##and the definition ends in ":"
def square_x(x):
    out = x * x
    return out

square_x(40)

Let's look a bit more closely at how the `square_x` function is written. It begins with the word `def` (short for define) followed by the name of the function, a variable named in parenthesis, and a colon. The variable is the input to the function. The colon is also a necessary part of the function definition, and it begins the code block that defines what the function does.

The rest of the function consists of this code block. In Python, this block must be indented with either tabs or spaces. The last line in the block contains the word 'return' followed by a variable name. This variable is the output of the function.

## Exercise

Make a new function called `cube_y` that takes `y` as an input, and returns the cube of `y`. 

Run `cube_y(2)` to test out your function.

In [ ]:
## space for your answer here



For the most part, we actually will want multiple inputs to our function, so we can do this by supplying more inputs to our function interface.

In [ ]:
def mult_xy(x, y):
    out = x * y
    return out

mult_xy(10, 5)

## 2.4 Grouping

Let's get back to our `DataFrame`. We'll define a function and use it using `groupby()`.  `groupby()` will group a `DataFrame` by one or more columns, and let you iterate through each group. 

In [ ]:
group_mouse = mousedata.groupby(['Sex'])

In [ ]:
##return only those mice with Sex = "M"
group_mouse.get_group('M')

What can you do with `groupby()`? One way to use it is to get *aggregate measures* based on group. For example, if we wanted to get the mean weight by gender, we can use the `apply` method on our data frames to return this. First we define a simple function called `mean_x` that returns the mean (we could have just used `np.mean` here, but it makes the code a little easier to understand).  

In [ ]:
def mean_x(x):
    return np.mean(x)

Then we can use the `apply()` function to get the mean by sex.

In [ ]:
mousedata.groupby(['Sex']).apply(mean_x)

Note the the `apply()` function takes a function as input and applies it to every element, in this case male and female weights.

**Question**: why did the `groupby` only return `Weight`? Does it make sense to do `mean_x` on our `Strain` variable?

## Exercise

Define a function to calculate the standard deviation (in numpy the function you need will be called `np.std`) and apply it to return the standard deviation of weights by `Strain`.

In [ ]:
## Space for your answer here.

## 2.5 Plotting

Let's look at some ways to visualize our `DataFrame`. We are going to use a module called `seaborn` to do our plotting, because the default plot options are pretty good, so we have to do less customization of our plots. In the next day, we'll see the module that `seaborn` is based on, `matplotlib`. 

Let's just plot the distribution of weights as a histogram. How many bins does our histogram have?

In [ ]:
##we need this line in our notebook to make matplotlib/seaborn work with Jupyter
%matplotlib inline

In [ ]:
# Histogram of weights
sns.distplot(mousedata.Weight)

## Exercise

Look up the help for `sns.distplot`. Note that there is a long list of input variables that are set as equal to `None` or `False`. This means that these are optional input variables that, unless defined, will run at their default definitions.

To practice utilizing these optional inputs, change the number of bins to 40 in the plot.

In [ ]:
help(sns.distplot)

## Space for your answer here.



## 2.6 Boxplots

Boxplots are super useful for looking at grouped means. Here we use the `sns.boxplot` function and group by `Sex`.

In [ ]:
# Boxplot
sns.boxplot(x = "Sex", y="Weight", data=mousedata)

# Set title with matplotlib
plt.title('Mouse weight by sex')
plt.show()

## Exercise 

Create boxplots showing the weight data measured from the 2 different strains, B6 and D2. Make sure to add a title to your plot, such as "Weight by Strain".

In [ ]:
##Space for your answer here


## 2.7 Faceting

Faceting is one of the most powerful ways of exploring data. For example, we can see whether there is a `Strain` by `Sex` effect by producing *conditional* boxplots, which separate the data out by condition.

Here we plot the distribution of weights conditioned on two variables: `Strain` and `Sex`. Note that these conditions are categorical variables, not numeric variables. 

One thing to notice is that we are passing in arguments to the `distplot` function within `g.map` as arguments to `g.map`. 

In [ ]:
##use sns.FacetGrid to define our facets
g = sns.FacetGrid(mousedata, col="Strain", row="Sex")
##then we can define our plots
g = g.map(sns.distplot, "Weight", bins = 30)

# A more complicated example

We're going to do much more manipulation and visualization with `pandas` using data taken from Imaris. Imaris is image analysis software with many sophisticated functions. Below is a confocal image taken of inner hair cells stained with antibodies against *CtBP2* (a pre-synaptic ribbon marker), *GluR2* (a post-synaptic receptor) and *MyosinVIIA* (which labels the entire hair cell). There are three color channels (red, green, and blue) which indicate the intensity of the staining for *CtBP2*, *GluR2* and *MyosinVIIa*, respectively.


## The data

Up to 25 auditory nerve fibers synapse onto individual inner hair cells in
normal-hearing individuals. However, these synapses can be permanently lost due
to aging, exposure to noise or ototoxic drugs.  In experiments that study
hearing loss, we need a way of quantifying the number of synapses per inner
hair cell.

One approach is to dissect the cochlea out of the experimental animals and use
whole-mount immunohistochemistry to label the tissue with antibodies for
pre-synaptic ribbons (CtBP2), post-synaptic receptors (GluR2) and cytoskeleton
(Myosin VIIa). The distribution of these proteins can be captured by taking a series of
two-dimensional images at various depths in the tissue.  These images are then
"stacked" to create a three-dimensional image known as a Z-stack (since the
third dimension is commonly referred to as the Z-axis).

<table>
	<body>
		<tr>
			<td>1A. CtBP2 (pre-synaptic ribbon)</td>
			<td>1B. GluR2 (post-synaptic glutamate receptor)</td>
		</tr>
		<tr>
			<td><img src="../day-4/data/CtBP2.png" /></td>
			<td><img src="../day-4/data/GluR2.png" /></td>
		</tr>
	</body>
</table>

## The problem

A functional inner hair cell synapse requires both a pre-synaptic ribbon and a
post-synaptic glutamate receptor. The next step in our analysis is to determine
whether each CtBP2 puncta is near a GluR2 label. 

This dataset was analyzed using Imaris to identify all CtBP2 puncta (white dots
in fig. 2a). If you look closely at the composite (fig. 2b), you'll see that
not all puncta have a glutamate receptor patch next to them (fig. 2b)! We
should not be counting these for the purpose of analysis. So, we need to find a
way to detect these false hits and eliminate them.

<table>
	<body>
		<tr>
			<td>A. CtBP2 puncta</td>
			<td>B. CtBP2 puncta overlaid on GluR2</td>
		</tr>
		<tr>
			<td><img src="../day-4/data/CtBP2+points.png" /></td>
			<td><img src="../day-4/data/CtBP2+GluR2+points.png" /></td>
		</tr>
	</body>
</table>

One approach is to extract a fixed volume around each CtBP2 puncta (e.g., a 1um
cube) and quantify the amount of GluR2 label in the volume. But, we don't know
very much about the format of the data. We need to do a little exploration first.

We used Imaris to detect all the "spots" in the CtBP2 (red) channel and compute some statistics about these spots. We've extracted the statistics file from the Imaris file into `csv` format just to make things easier. Just know that there are routines to extract this information from the file. Today we will practice exploring and visualizing the Imaris data and on Day 4 we will return to the question of how to quantify the amount of GluR2 label in a fixed volume around each CtBP2 puncta.

Ok, let's load in the statistics for the CtBP2 puncta that were calculated by Imaris.

In [ ]:
point_stats = pd.read_csv("data/points_statistics.csv")

## 2.6 Exploring the Imaris Statistics

Because this data file was automatically generated by Imaris, we first need to figure out how it is organized.

We can start taking a look at the first few rows of our summary table using `point_stats.head()`. In general, this is a really good practice to get into. Sometimes our data may have a header or not, and we may have loaded the data incorrectly.

In [ ]:
##Show first few rows
point_stats.head()

What are some things we notice? Well, there appear to be some data that describe the entire sample (such as "Total Number of Spots") as well as data for localized points identified by Imaris in the red channel (such as "Area").  

In [ ]:
##show last few rows
point_stats.tail()

In [ ]:
##show dimensions of data frame
point_stats.shape

We can also see that attributes for the various traits describing a given spot (such as "Area" and "Volume") are not columns, but rather listed under the categorical column "Name." If we are curious to see this full list of names, use the unique() function: 

In [ ]:
point_stats.Name.unique()

Looking back to the first few rows of data, it appears that ID_Object of -1 designates statistics that describe the entire sample. Let's confirm by viewing all rows with ID_Object of -1:

In [ ]:
##we will talk more about the point_stats["ID_Object"]==-1 notation in
##section 
point_stats[point_stats["ID_Object"]==-1]

Let's look at all of the statistics that were collected for a single spot identified by Imaris, starting with the first:

In [ ]:
point_stats[point_stats["ID_Object"]==0]

Let's look at the raw data for Diameter of spots in the X dimension ("Diameter X"):

In [ ]:
point_stats[point_stats["Name"]=="Diameter X"].head(20)
#OR
point_stats[point_stats["ID_StatisticsType"]==237].head(20)

## 2.7 Indexing

 We use `loc()` when we want to access things by *name* or *index* (it's position in our `DataFrame`). 

Remember when we were slicing the strings in Day 1? We use a very similar notation to access rows and columns in our `DataFrame`. Because the `DataFrame` has two dimensions, we need to specify two sets of positions to `loc`: column positions and row positions. We pass this on as a list of `row_positions` and `column_positions`.

```
point_stats.loc([row_positions, column_positions])
```

Let's use this notation to grab only the columns we want. Say we only want to view the `ID_Object`, `Value`, and `Name` columns from our `DataFrame`.

Here we define a list called `colnames` and then pass as an argument into `.loc()`. Since we want all of the associated rows, we can use the `:` as an input to `row_positions`. `:` says, return all the values for the rows.

In [ ]:
colnames = ["ID_Object", "Value", "Name"]
col_selected_data = point_stats.loc[:,colnames]
selected_data.head(10)

Say we want to refer to these by the position in the dataframe. For example, say we want the first, second, and fifth columns in our dataset. Remembering that indexing always starts at 0:

In [ ]:
col_index = [0,1,4]

col_selected_data = point_stats.iloc[:,col_index]

col_selected_data.head(10)

## Exercise

Use our `mean_x` function to return the mean `Intensity Max X` across the dataset.

In [ ]:
## Space for your answer here



## 2.7 Pivoting

Now let's create a DataFrame that is more intuitive in terms of viewing the statistics Imaris has collected for each identified spot in the red channel. We will call this DataFrame `point_stats_matrix`. To do this, use the `pivot()` function, which reshapes data based on column values. This function is extremely useful in transforming data from *long* format to *wide* format. 

The `pivot` method takes three arguments: `index`, which you can think of as being the rows of the data, `columns`, which specify what columns should exist in the data, and `values`, which are the actual numerical values we want in each Cell.

In [ ]:
point_stats_matrix = point_stats.pivot(index='ID_Object', columns='Name', values='Value')
point_stats_matrix.head()

Remember that the statistics for the entire data set (including "Number of spots per time point" and "Total number of spots") have an ID_Object of -1. Let's remove this row:

In [ ]:
point_stats_matrix = point_stats_matrix.drop(-1)
point_stats_matrix.head(20)

In [ ]:
point_stats_matrix.describe()

## 2.8 Plotting our DataFrame for further exploration

Next let's try some simple visualization, starting with a histogram of area measurements for the spots. Remember that we already imported `seaborn` and `matplotlib` above in order to use plotting functions contained in these modules, so we don't need to import them again before using the functions below. As a reminder, when we imported, we abbreviated the `seaborn` module as `sns`. 

In [ ]:
sns.distplot(point_stats_matrix.Area)

How about a boxplot of Area values?

In [ ]:
sns.boxplot(y="Area", data=point_stats_matrix)

## Exercise

Take a look at the help for `lmplot` below and make a scatterplot comparing `Intensity Max Z` on the x axis against `Volume` on the y axis. 

In [ ]:
help(sns.lmplot)

?sns.kdeplot


In [ ]:
sns.kdeplot(x='Intensity Max Z', y='Volume', data=point_stats_matrix)
sns.lmplot(x='Intensity Max Z', y='Volume', fit_reg=False, data=point_stats_matrix)


## 2.9 Filtering

Next we will discuss filtering. The scatterplot you created in the exercise above shows a smattering of points with an unsually large volume. Perhaps we decide that we don't trust that these are isolated points and therefore should exclude these outliers from our dataset. To do this, we will create a DataFrame named `filtered_points` that only includes spots with a volume less than 0.8. 

We do this by first defining a *mask* that will only return the data we are interested in. Our *mask* will remove those rows in the data that do not meet our criterion. 

In [ ]:
mask = point_stats_matrix.Volume <= 0.8
filtered_points=point_stats_matrix[mask]
filtered_points.head()

Now let's create another scatterplot to confirm that our filter worked:

In [ ]:
#space for new scatterplot

sns.lmplot(x='Intensity Max Z', y='Volume', fit_reg=False, data=filtered_points)

**Question**: what is the output of `point_stats_matrix.Volume <= 0.8`? Try it out by running the below cell. 
    
How does this output help us to select the rows we want out of `point_stats_matrix`? (Hint: Think about what `True` and `False` mean in this context. Does it mean we want to return that value or not?)

In [ ]:
point_stats_matrix.Volume <= 0.8

## Exercise

Filter the `points_stats_matrix` dataset to have `Intensity Center X > 10000` and assign the result to `psm10000`. (Because of the spaces in the variable names, you will have to use points_stats_matrix['Intensity Center X'] to access the column).

Re-do the scatter plot of X and Y Intensity Centers to confirm that your filtering worked. 

In [ ]:
##space for your answer here.

## 2.10 Merging DataFrames

Another powerful way of manipulating data in Pandas is to combine two `DataFrame`s together. In fact, we actually merged two `DataFrame`s together to get the original `point_stats_matrix` `DataFrame`. Let's load in the two separate `DataFrame`s so we can see how we did this. Our new relative path is `data/raw-data/`, since that is where the unmerged data is.

In [ ]:
## we have to use pd.read_table instead of pd.read_csv because the data
## is tab delimited
points_value = pd.read_table("data/raw-data/points-statistics-value.txt", header=None)
points_value.head()

Since there are no column names in the file, we have to add them manually by setting the `columns` property. How does `.columns` know which columns to rename to our new column names? We pass it a list whose values are the new column names:

In [ ]:
new_col_names = ["ID_Time","ID_Object","ID_StatisticsType","Value"]
points_value.columns = new_col_names
points_value

Now let's open the `points-statistics-type.txt` as a table:

In [ ]:
points_type = pd.read_table("data/raw-data/points-statistics-type.txt", header=None)

new_col_names = ["ID_StatisticsType", "ID_Category", "ID_FactorList", "Name", "Unit"]
points_type.columns = new_col_names

points_type.head()

Huh. We notice that both `points_type` and `points_value` have a column called `ID_StatisticsType`. Can we use this to combine the two tables? Yes! There's a function called `.merge()` in `pandas` that will let us do it.

Note we're using a function from `pd`, and not from the `DataFrame` to do this. `pd.merge` takes four arguments:

1. `left` - the left side table, which is `points_value`.
2. `right` - the right side table, which is `points_type`.
3. `left_on` - the column name in the left table that we want to merge on, which is `"ID_StatisticsType"`.
4. `right_on` - the column name in the right table that we want to merge on, which is `"ID_StatisticsType"`.

In [ ]:
merged_table = pd.merge(left=points_value, right=points_type, 
         left_on = "ID_StatisticsType", right_on="ID_StatisticsType")

merged_table.head()

## 2.11 Getting data out

What if you want to save the `point_stats_matrix` DataFrame as its own csv file? Try running the code below. Where did it write the dataset?

In [ ]:
point_stats_matrix.to_csv("data/point_stats-mod.csv")

There is also support for reading and writing Excel files if you need it: http://pandas.pydata.org/pandas-docs/stable/io.html#excel-files

## 2.12 Discussion questions

1. What is the difference between `groupby()` and filtering? When would you want to use each of these?
2. What is the difference between filtering and indexing? 

## 2.13 What you learned today

Congrats for getting this far! You have seen lots of features of Pandas and Seaborne that let you manipulate the data and visualize it. 

1. `group_by`
2. Filtering
3. Boxplots and Scatterplots
4. Faceting
5. Pivoting data

There's a lot more about `DataFrames` to learn! Here's some good resources to learn even more:

https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python